# Skin Cancer Detection using Machine Learning and Computer Vision

## Project Description

This project aims to develop a robust and accurate skin cancer detection system using machine learning and computer vision techniques. By leveraging a dataset of skin lesion images, the system will be trained to classify images into different categories of skin cancer, aiding in early diagnosis and treatment. The project involves data preprocessing, feature extraction, model training, and evaluation to ensure high performance and reliability.

In [ ]:
# Étape 1 : Installer les bibliothèques nécessaires
!pip install tensorflow
!pip install matplotlib
!pip install numpy

     ---------------------------------------- 0.0/59.7 kB ? eta -:--:--
     ------ --------------------------------- 10.2/59.7 kB ? eta -:--:--
     -------------------------------------- 59.7/59.7 kB 526.6 kB/s eta 0:00:00
     ---------------------------------------- 0.0/48.7 kB ? eta -:--:--
     ---------------------------------------- 48.7/48.7 kB 2.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/390.3 MB ? eta -:--:--
   ---------------------------------------- 0.8/390.3 MB 16.6 MB/s eta 0:00:24
   ---------------------------------------- 2.0/390.3 MB 21.9 MB/s eta 0:00:18
   ---------------------------------------- 3.7/390.3 MB 26.3 MB/s eta 0:00:15
    --------------------------------------- 6.3/390.3 MB 33.8 MB/s eta 0:00:12
   - -------------------------------------- 10.3/390.3 MB 43.7 MB/s eta 0:00:09
   - -------------------------------------- 15.7/390.3 MB 93.9 MB/s eta 0:00:04
   -- ------------------------------------ 21.1/390.3 MB 108.8 MB/s eta 0:00


[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, Input, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array


In [ ]:
# Chargement du modèle EfficientNet-B0 préentraîné
# Dimensions de l'image (selon votre prétraitement)
IMAGE_SIZE = (100, 100, 3)  # 100x100 pixels, RGB

# Chargement du modèle préentraîné
base_model = EfficientNetB0(
    include_top=False,      # Exclure la dernière couche de classification
    weights="imagenet",     # Utiliser les poids préentraînés sur ImageNet
    input_shape=IMAGE_SIZE  # Taille des images d'entrée
)

# Geler les couches du modèle de base pour conserver les poids préentraînés
base_model.trainable = False

In [ ]:
# Entrée du modèle
inputs = Input(shape=IMAGE_SIZE)

# Passage des images dans EfficientNet-B0
x = base_model(inputs, training=False)
x = Flatten()(x)  # Aplatir les caractéristiques extraites

# Ajout de couches fully connected pour la classification
x = Dense(128, activation="relu")(x)  # Couche dense
x = Dropout(0.5)(x)                   # Dropout pour régularisation
outputs = Dense(1, activation="sigmoid")(x)  # Sigmoid pour une classification binaire

# Définition du modèle final
model = Model(inputs, outputs)

In [ ]:
model.compile(
    optimizer=Adam(learning_rate=0.001),  # Optimiseur Adam
    loss="binary_crossentropy",          # Fonction de perte pour la classification binaire
    metrics=["accuracy"]                 # Suivi de la précision
)

# Résumé du modèle
model.summary()

In [ ]:
# Visualisation de l'architecture du modèle
plot_model(model, show_shapes=True, to_file="efficientnetb0_model.png")

In [ ]:
def predict_skin_cancer(image_path, metadata):
    # Charger et prétraiter l'image
    img = load_img(image_path, target_size=(100, 100))
    img_array = img_to_array(img) / 255.0
    img_array = tf.expand_dims(img_array, axis=0)

    # Préparer les métadonnées
    metadata = tf.expand_dims(metadata, axis=0)

    # Faire la prédiction
    prediction = model.predict([img_array, metadata])

    # Interprétation
    if prediction[0] > 0.5:
        return f"Maligne (probabilité {prediction[0][0]:.2f})"
    else:
        return f"Bénigne (probabilité {1 - prediction[0][0]:.2f})"

# Exemple d'utilisation
print(predict_skin_cancer("path_to_your_image.jpg", [30, 5.2, 1]))
